# Youtube Adview Prediction

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## EDA

In [ ]:
df=pd.read_csv('train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

Removing F values from dataset

In [ ]:
df=df[df.views!='F']
df=df[df.likes!='F']
df=df[df.dislikes!='F']
df=df[df.comment!='F']

Converting them to numeric

In [ ]:
df['views']=pd.to_numeric(df['views'])
df['likes']=pd.to_numeric(df['likes'])
df['dislikes']=pd.to_numeric(df['dislikes'])
df['comment']=pd.to_numeric(df['comment'])
df['published']=pd.to_datetime(df['published'])

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
vidid_col=df['vidid']
vidid_col

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

df['category']=LabelEncoder().fit_transform(df['category']).astype('int64')
df['vidid']=LabelEncoder().fit_transform(df['vidid']).astype('int64')

In [ ]:
df.head()

## Conveting duration to readable format

In [ ]:
import time
import datetime

In [ ]:
sample=df['duration'][0]
sample[2:]

In [ ]:
def check(x):
    y = x[2:]
    h = ''
    m = ''
    s = ''
    duration = ''
    P = ['H','M','S']
    for i in y:
        if i not in P:
            duration+=i
        else:
            if(i=="H"):
                h = duration
                duration = ''
            elif(i == "M"):
                m = duration
                duration = ''
            else:
                s = duration
                duration = ''
                
    if(h==''):
        h = '00'
    if(m == ''):
        m = '00'
    if(s==''):
        s='00'
    
    bp = h+':'+m+':'+s
    return bp

In [ ]:
a='PT7M37S'
print(check(a))

In [ ]:
train=pd.read_csv("train.csv")
mp = pd.read_csv("train.csv")["duration"]
print(mp)
time = mp.apply(check)
print(time)

In [ ]:
def func_sec(time_string):
    h, m, s = time_string.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

time1=time.apply(func_sec)

In [ ]:
df['duration']=time1
df.head()

In [ ]:
df.info()

## Visualization

In [ ]:
plt.hist(df['category'])
plt.show()

In [ ]:
plt.plot(df['adview'])
plt.show()

Removing outlier

In [ ]:
df=df[df.adview<2000000]

In [ ]:
plt.plot(df['adview'])
plt.show()

In [ ]:
corr=df.corr()
sns.heatmap(corr,mask=np.zeros_like(corr,dtype=np.bool),annot=True)